# This code file is to prepare for the data that we are going to use for the analysis

## What Files Are Given to Start with:

**us_cities_by_state_SEPT.2023.csv**: the names/urls of the articles we would be working with. This data is scraped from [this WikiPedia page](https://en.wikipedia.org/wiki/Category:Lists_of_cities_in_the_United_States_by_state) and the scraped result could be found [here](https://drive.google.com/file/d/1khouDmMaZyKo0y5WkFj4lu7g8o35x_98/view?usp=sharing)

**NST-EST2022-ALLDATA.csv**: this data is obtained from [here](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html). I chose the full dataset instead of the excel file because the excel file is not in a table. I took out only the names of states and the estimated population of the states for the analysis

**US States by Region in Table - US Census Bureau.csv**: the original dataset *US States by Region - US Census Bureau.csv* comes from [this file](https://docs.google.com/spreadsheets/d/14Sjfd_u_7N9SSyQ7bmxfebF_2XpR8QamvmNntKDIQB0/edit?usp=sharing)

## What Tools Used in this Project:

**Wikimedia**:  to extract the page information of articles. [Sample code used in this project comes from here](https://drive.google.com/file/d/15UoE16s-IccCTOXREjU3xDIz07tlpyrl/view?usp=sharing)

**ORES model**: provides the peer reviewed score of each article based on the quality. [Sample code used in this project comes from here](https://drive.google.com/file/d/17C9xsmR9U3lJeD52UTbAedlHDetwYsxs/view?usp=sharing)

## What Steps Are Covered in this Code File:

**Scraping the page information of articles** - using the *Wikimedia API*

**Obtaining the assessed quality score for each article** - using the ORES model. A second attempt was performed after noting down which articles failed to retrieve its score, and only one failed again to retrieve its score on the second try which was removed. It is also worth noticing that there exist duplicates of rows of data which should be removed in order to perform the analysis

**Combining the datasets** - the previous steps result in a DataFrame with columns of article name, state, review id and score. Population (which we could obtain from the *NST-EST2022-ALLDATA.csv*) and regional data (which is in *US States by Region in Table - US Census Bureau.csv*) are required in the final dataset, which should contain in total 6 columns

### Scraping using Wikimedia

In [ ]:
# !pip install openpyxl --upgrade
import pandas as pd
import json, time, urllib.parse
import requests
import warnings
warnings.simplefilter('ignore')

pop_est_2022 = pd.read_csv('NST-EST2022-ALLDATA.csv')
cities_by_state = pd.read_csv('us_cities_by_state_SEPT.2023.csv')

### Obtained the list of the titles of the articles whose page information we are looking for

In [ ]:
article_titles = list(cities_by_state['page_title'])

In [ ]:
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'hww1999@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests

ARTICLE_TITLES = article_titles

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


In [ ]:
def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    # make the request
    try:
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


### Handled exceptions here to prevent failure to retrieve

One intermediary data file generated here for backup purpose.

In [ ]:
cities_by_state_articles = {"articles":[]}
for i in range(len(ARTICLE_TITLES)):
    print("Getting page information for: ",ARTICLE_TITLES[i])
    views = request_pageinfo_per_article(ARTICLE_TITLES[i])
    try:
        pid = list(views['query']['pages'].keys())[0]
        cities_by_state_articles["articles"].append(views['query']['pages'][pid])
    except Exception as e:
        print(e)
with open("cities_by_state.json", "a") as outfile:
        outfile.write(json.dumps(cities_by_state_articles,indent=4))

In [ ]:
tmp = pd.read_json('cities_by_state.json')['articles']
tmp = pd.json_normalize(tmp)

In [ ]:
df_eval = tmp[['pageid', 'title', 'lastrevid', 'talkid']]

### Evaluation using ORES model

In [ ]:
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (60.0/5000.0)-API_LATENCY_ASSUMED

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2023",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}

#
#    This is a template for the parameters that we need to supply in the headers of an API request
#

REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = {'Abbeville, Alabama': 1171163550}
# { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}



### Personal access token is saved in a local file

In [ ]:
# input personal information
# access token saved as a local file

token_info = pd.read_csv('WikiAPIToken.csv')
ACCESS_TOKEN = token_info['Access token'][0]
USERNAME = "" # email address

In [ ]:
def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


### Below is the first attempt with 107 failures

In [ ]:
score_pred = []
from tqdm import tqdm
for i in tqdm(range(df_eval.shape[0])):
    
    article_title = df_eval.iloc[i]['title']
    article_revid = df_eval.iloc[i]['lastrevid']
    
#     print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {article_revid:d}")
    try:
    
        score = request_ores_score_per_article(article_revid=int(article_revid),
                                               email_address=USERNAME,
                                               access_token=ACCESS_TOKEN)

        pred = score['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction']
#         print(article_title + ': ' + pred)
        score_pred.append(pred)
    except Exception as e:
        print(e)
        print(article_title + ': Unknown')
        score_pred.append('REQUEST FAIL')

### The second intermediary dataset generated for back up purpose

In [ ]:
df_eval['score_pred'] = pd.Series(score_pred)
df_eval.to_csv('score_first_attemp.csv', index=False)
df_eval = pd.read_csv('score_first_attemp.csv')

### At this time, I realized I forgot to include the state column from the given file, so added in here

In [ ]:
df_eval['state'] = cities_by_state['state']

In [ ]:
df_failed_first_attempt = df_eval[df_eval['score_pred']=='REQUEST FAIL'][['title', 'lastrevid']]
df_failed_first_attempt = df_failed_first_attempt.reset_index()

### This attempt fixed 106 failed retrievals among 107

In [ ]:
second_attempt = []
for i in range(df_failed_first_attempt.shape[0]):
    
    article_title = df_failed_first_attempt.iloc[i]['title']
    article_revid = df_failed_first_attempt.iloc[i]['lastrevid']
    
#     print(f"Getting LiftWing ORES scores for '{article_title}' with revid: {article_revid:d}")
    try:
    
        score = request_ores_score_per_article(article_revid=int(article_revid),
                                               email_address=USERNAME,
                                               access_token=ACCESS_TOKEN)
#         print(score)
        second_attempt.append(score['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction'])
#         pred = score['enwiki']['scores'][str(article_revid)]['articlequality']['score']['prediction']
#         print(article_title + ': ' + pred)
#         second_attempt.append(pred)
    except Exception as e:
#         print(e)
#         print(article_title + ': Unknown')
        second_attempt.append('REQUEST FAIL')
df_failed_first_attempt['score'] = pd.Series(second_attempt)

In [ ]:
# df_failed_first_attempt.reset_index()
# df_failed_first_attempt['score'] = pd.Series(second_attempt)
df_failed_first_attempt[df_failed_first_attempt['score']=='REQUEST FAIL']

In [ ]:
for i in range(df_failed_first_attempt.shape[0]):
    currtitle = df_failed_first_attempt.iloc[i]['title']
    currrevid = df_failed_first_attempt.iloc[i]['lastrevid']
    currscore = df_failed_first_attempt.iloc[i]['score']
    idx = df_eval[df_eval['title']==currtitle][df_eval['lastrevid']==currrevid].index
    df_eval.loc[idx,['score_pred']] = currscore

### Dropped duplicates at this point

In [ ]:
df_eval = df_eval.drop_duplicates()
df_eval.shape

In [ ]:
df_eval = df_eval.reset_index()

### We found that the same article but with a different review id is there with score already, so removed the one that failed

In [ ]:
df_eval[df_eval['score_pred']=='REQUEST FAIL']

In [ ]:
df_eval[df_eval['title']=='Auburn, Alabama']

### The third intermediary data file generated which should overwrite the score_first_attemp.csv given that it only filled in the scores that failed to retrieve at the first try

In [ ]:
df_eval = df_eval.drop([461])
df_eval = df_eval.reset_index()
df_eval.to_csv('updated_score.csv', index=False)

### Combining the datasets

In [ ]:
df_eval = pd.read_csv('updated_score.csv')
df_eval = df_eval.reset_index()

In [ ]:
df_eval = df_eval.drop(columns=['level_0', 'index'])
df_eval

### Noted there is no Connecticut or Nebraska from the original scraped dataset

In [ ]:
len(df_eval['state'].unique()) # no Connecticut or Nebraska

In [ ]:
df_eval['state'].unique() # need to replace '_' with space, and 'Georgia_(U.S._state)' with 'Georgia'

In [ ]:
df_eval['state'] = df_eval['state'].str.replace('Georgia_(U.S._state)','Georgia', regex=False)
df_eval['state'] = df_eval['state'].str.replace('_',' ', regex=False)

### We only need the NAME of the states and the POPESTIMATE2022 for the estimated population in 2022 for each state

In [ ]:
pop_est_2022.columns

In [ ]:
pop_2022 = pop_est_2022[['NAME', 'POPESTIMATE2022']]
pop_2022 = pop_2022.rename(columns={"NAME": "state", "POPESTIMATE2022": "population"})

### Join on the state column to obtain the population field

In [ ]:
df_pop = pd.merge(df_eval, pop_2022, on=['state'])
df_pop

### Find out the region each state belongs to and join on state column again

In [ ]:
states_by_region = pd.read_csv('US States by Region in Table - US Census Bureau.csv')

In [ ]:
states_by_region = states_by_region.rename(columns={"REGION": "regional_division", "STATE": "state"})
states_by_region = states_by_region.drop(columns=['DIVISION'])

In [ ]:
df_final = pd.merge(df_pop, states_by_region, on=['state'])
df_final

In [ ]:
df_final = df_final.rename(columns={"title": "article_title", "lastrevid": "revision_id", "score_pred": "article_quality"})

In [ ]:
df_final.to_csv('wp_scored_city_articles_by_state.csv', index=False)